In [113]:
using JuMP               # To write the optimisation models
using Cbc, Clp           # Solvers 
using Test               # Testing package
using JLD2;              # File I/O

# Homework 4

## Problem description

We need to fulfil the demand of clients using different servers. The demand and set of clients are unknown when making the decision on which servers to use. Consider the parameters:
- $C_j$ - cost of installing server $j$
- $P_s$ - probability of scenario $s$
- $F$ - cost of unmet demand (same for all clients $i$, servers $j$ and scenarios $s$)
- $Q_{ij}$ - benefit per one unit of demand of client $i$ served by server $j$
- $V$ - maximum allowed number of servers
- $D_{is}$ - demand of client $i$ in scenario $s$
- $U$ - maximum capacity of a server (same for all servers and scenarios)
- $H_{is}$ - a binary variable with value 1 if client $i$ is active in scenario $s$, i.e., the demand $D_{is}$ has to be fulfilled

Let the variables be

- $x_j$ - binary variable with value 1 if server $j$ is made available, i.e., built or installed
- $y_{ijs}$ - the proportion of demand $D_{is}$ fulfilled by server $j$. The total demand of $i$ served by $j$ is thus $y_{ijs} \times D_{is}$
- $z_{js}$ - capacity shortage for server $j$. If demand is not met otherwise, any server $j$ can procure emergency capacity at a price $F$.


The model is then given by:

\begin{align*}
    \min_{x_j, z_{js}, y_{ijs}} & \sum_{j \in J} C_j x_j + \sum_{s} P_s \left( - \sum_{i \in I,j \in J}Q_{ij}D_{is}y_{ijs} + \sum_{j \in J} Fz_{js} \right) \\
    \text{s.t.: } & \sum_{j \in J} x_j \leq V   & (t)\\
    & \sum_{i \in I} D_{is}y_{ijs} - z_{js} \leq Ux_j, \forall j \in J, s \in S  & (u_{js})\\
    & \sum_{j \in J} y_{ijs} = H_{is}, \forall i \in I, s \in S  & (v_{is}) \\
    & x_j \in \{0,1\}, \ \forall j \in J \\
    & y_{ijs} \geq 0, \ \forall i \in I, \forall j \in J, \forall s \in S \\
    & z_{js} \geq 0, \ \forall j \in J, \forall s \in S
\end{align*}

, where $t$, $u_{js}$, and $v_{is}$ are the dual variables related to the constraints in the model.

In [114]:
struct Instance
    # sets
    I  # Set of clients
    J  # Set of servers
    S  # Set of scenarios
    # Parameters 
    V  # Max number of servers
    P  # Scenario probabilities
    H  # 1 if client requires service
    C  # Cost of locating server at j
    F  # Cost of unmet demand
    D  # Demand in location i served from server j
    Q  # Benefit per unit of demand served
    U  # Maximum server capacity (same for all servers)
    loc_i # Coordinates of clients
    loc_j # Coordinates of servers
end

In [115]:
f = jldopen("hw4_ins.jld2")
ins = nothing
try
    ins = f["ins"]
finally
    close(f)
end;

# Task 1: implementing the large-scale model

## Model construction 

In the following, the full model must be implemented and solved using Cbc.

In [116]:
function generate_full_problem(ins::Instance)
    I = ins.I 
    J = ins.J
    S = ins.S
    V = ins.V 
    P = ins.P
    H = ins.H
    C = ins.C
    F = ins.F
    D = ins.D
    Q = ins.Q
    U = ins.U
    # Write the full model in JuMP

    # Initialize model
    m = Model(Cbc.Optimizer)
    
    # TODO: add your code here
    @variable(m, x[J], Bin)
    @variable(m, y[I, J, S] >= 0)
    @variable(m, z[J, S] >= 0)

    @expression(m, cost_server_loc, sum(C[j] * x[j] for j in J))

    @expression(m, cost_extra_cap[s in S], sum(F * z[j, s] for j in J))
    @expression(m, benifit_supply[s in S], sum(Q[i, j] * D[i, s] * y[i, j, s] for i in I for j in J))
    @expression(m, exp_benefit, sum(P[s] * (cost_extra_cap[s] - benifit_supply[s]) for s in S))

    @objective(m, Min, cost_server_loc + exp_benefit)

    @constraint(m, number_of_server, sum(x[J]) <= V)
    @constraint(m, server_capacity[j = J, s = S], sum(D[i, s] * y[i, j, s] for i in I) <= U * x[j] + z[j, s])
    @constraint(m, affected_loc[i = I, s = S], sum(y[i, j, s] for j in J) == H[i, s])
    
    # Return the generated model
    return m
end

generate_full_problem (generic function with 1 method)

In [117]:
fullmodel = generate_full_problem(ins)
set_silent(fullmodel)
optimize!(fullmodel)

In [118]:
# Examine the solutions
@show x_bar = Int.(round.(value.(fullmodel[:x]).data))
@show opt_z = sum(value.(fullmodel[:z]))
@show round(objective_value(fullmodel), digits = 0);

x_bar = Int.(round.(value.(fullmodel[:x]).data)) = [0, 0, 0, 1, 1]
opt_z = sum(value.(fullmodel[:z])) = 0.0
round(objective_value(fullmodel), digits = 0) = -1327.0


# Task 2: Benders decomposition

## Benders main

Formulate the initial main problem for the decomposition. Use a single variable $\theta$ for representing the subproblem value.

The model is then given by:

\begin{align*}
    \min_{x_j, \theta} & \ \sum_{j \in J} C_j x_j + \theta \\
    \text{s.t.: } & \sum_{j \in J} x_j \leq V   & (t)\\
    & x_j \in \{0,1\}, \ \forall j \in J \\
    & \theta \in \mathbb{R}
\end{align*}

In [132]:
## Benders decomposition

## Generates the main problem
function generate_main(ins)
    
    J = ins.J
    V = ins.V
    C = ins.C
     
    main = Model(Cbc.Optimizer)
    set_silent(main)
    
    # TODO: add your code here
    @variable(main, x[J], Bin)
    @variable(main, θ)

    @objective(main, Min, sum(C[j] * x[j] for j in J) + θ)

    @constraint(main, number_of_server, sum(x[J]) <= V)

    return main  
end

generate_main (generic function with 1 method)

In [133]:
# Solve the main problem
function solve_main(ins, main)
    optimize!(main)
    return value.(main[:x]), value(main[:θ]), objective_value(main)    
end

solve_main (generic function with 1 method)

## Subproblem

Formulate the primal subproblem with corresponding objective value represented by the variable $\theta$ in the main problem. The primal subproblem is not used in the decomposition algorithm, but you will use it to test your implementation of the dual subproblem. It might also be easier to start by formulating the primal problem and then work from there to obtain the its dual formulation.

The primal subproblem is formulated as:

\begin{align*}
    \min_{z_{js}, y_{ijs}} & \sum_{s} P_s \left( - \sum_{i \in I,j \in J}Q_{ij}D_{is}y_{ijs} + \sum_{j \in J} Fz_{js} \right)\\
    \text{s.t.: } & \sum_{i \in I} D_{is}y_{ijs} - z_{js} \leq U \overline{x}_j, \ \forall j \in J, s \in S  & (u_{js})\\
    & \sum_{j \in J} y_{ijs} = H_{is}, \ \forall i \in I, s \in S  & (v_{is}) \\
    & y_{ijs} \geq 0, \ \forall i \in I, \forall j \in J, \forall s \in S \\
    & z_{js} \geq 0, \ \forall j \in J, \forall s \in S
\end{align*}

In [121]:
# Generate and solve the primal subproblem for a given x_bar. For test purposes only; if the dual is correct, the objective value of
# the dual subproblem must be the same as this.
function generate_and_solve_primal_subproblem(ins, x_bar)
    
    I = ins.I
    J = ins.J
    S = ins.S
    D = ins.D
    P = ins.P
    Q = ins.Q
    F = ins.F
    U = ins.U
    H = ins.H
    
    # set_silent works for Clp, and the subproblem should be an LP problem    
    sub = Model(Clp.Optimizer)
    set_silent(sub)

    # TODO: add your code here
    @variable(sub, y[I, J, S] >= 0)
    @variable(sub, z[J, S] >= 0)

    @expression(sub, cost_extra_cap[s in S], sum(F * z[j, s] for j in J))
    @expression(sub, benifit_supply[s in S], sum(Q[i, j] * D[i, s] * y[i, j, s] for i in I for j in J))
    @expression(sub, exp_benefit, sum(P[s] * (cost_extra_cap[s] - benifit_supply[s]) for s in S))

    @objective(sub, Min, exp_benefit)

    @constraint(sub, server_capacity[j = J, s = S], sum(D[i, s] * y[i, j, s] for i in I) <= U * x_bar[j] + z[j, s])
    @constraint(sub, affected_loc[i = I, s = S], sum(y[i, j, s] for j in J) == H[i, s])
    
    optimize!(sub)
    return objective_value(sub)
    
end

generate_and_solve_primal_subproblem (generic function with 1 method)

## Dual subproblem

Formulate the dual subproblem. Consider the dual variables indicated in the fullmodel as we are expecting you to use the same names. Hint: You can find the conversion rules for primal and dual problems in Lecture 5.

Formulation of the dual subproblem is thus derived as:

\begin{align*}
    \max_{u_{js}, v_{is}} & \sum_{j \in J, s \in S} (U \overline{x}_j) u_{js} + \sum_{i \in I, s \in S} H_{is} v_{is} \\
    \text{s.t.: } & D_{is} u_{js} + v_{is} \leq -P_s Q_{ij} D_{is}, \ \forall i \in I, j \in J, s \in S  & (y_{ijs}) \\
    & - u_{js} \leq P_{s} F, \ \forall j \in J, s \in S  & (z_{js}) \\
    & u_{js} \leq 0, \ \forall j \in J, \forall s \in S \\
    & v_{is} \in \mathbb{R}, \ \forall i \in I, \forall s \in S
\end{align*}

In [122]:
## Define Benders subproblem
function generate_and_solve_dual_subproblem(ins, x_bar)
    
    I = ins.I
    J = ins.J
    S = ins.S
    D = ins.D
    P = ins.P
    Q = ins.Q
    F = ins.F
    U = ins.U
    H = ins.H
    
    # set_silent works for Clp, and the subproblem should be an LP problem
    sub_dual = Model(Clp.Optimizer)
    set_silent(sub_dual)
    
    # TODO: add your code here
    @variable(sub_dual, u[J, S] <= 0)
    @variable(sub_dual, v[I, S])

    @objective(sub_dual, Max, sum(U * x_bar[j] * u[j,s] for j in J for s in S) + sum(H[i,s] * v[i,s] for i in I for s in S))

    @constraint(sub_dual, dual_y[i=I, j=J, s=S], D[i,s] * u[j,s] + v[i,s] <= - P[s] * Q[i,j] * D[i,s])    
    @constraint(sub_dual, dual_z[j=J, s=S], - u[j,s] <= P[s] * F)

    optimize!(sub_dual)
    
    u_bar = value.(sub_dual[:u])                     
    v_bar = value.(sub_dual[:v])                     
    opt_value = objective_value(sub_dual)
    
    return u_bar, v_bar, opt_value
end

generate_and_solve_dual_subproblem (generic function with 1 method)

## Benders cut

Formulate the Benders optimality cut. Remember to explain in your report why you only need to consider one type of cut.

Benders cut formulation:

\begin{align*}
    \theta \geq \sum_{j \in J, s \in S} (U x_j) \overline{u}_{js} + \sum_{i \in I, s \in S} H_{is} \overline{v}_{is}
\end{align*}

In [141]:
# Add the Benders cut, given current dual values
function add_benders_cut(ins, main, u_bar, v_bar)   
    
    U = ins.U
    H = ins.H
    I = ins.I
    J = ins.J
    S = ins.S
    
    x = main[:x]
    θ = main[:θ]
    
    @constraint(main,  
    # TODO: add your code here
    θ >= sum(U * x[j] * u_bar[j,s] for j in J for s in S) + sum(H[i,s] * v_bar[i,s] for i in I for s in S)
    )
    return main
end

add_benders_cut (generic function with 1 method)

### Testing the subproblem formulation

You can use the cell below to check whether your implementation of the subproblem is correct. For a fixed solution from the main problem, strong duality holds and thus these objective function values should match. We use `≈` which is equivalent to `approx()` to test whether the values are sufficiently close.

In [131]:
## Test that the primal and dual solutions are the same
(u,v,optval) = generate_and_solve_dual_subproblem(ins, x_bar)
obj = generate_and_solve_primal_subproblem(ins, x_bar)
@test optval ≈ obj

# C = ins.C
# J = ins.J
# obj_main = sum(C[j] * x_bar[j] for j in J)
# @show obj_main+optval

Test Passed
  Expression: optval ≈ obj
   Evaluated: -1460.3664495861512 ≈ -1460.3664495861508

## Benders decomposition algorithm

Here you will combine the functions you defined before into the complete algorithm. 

Some hints:
- You should add a cut before solving the main problem for the first time to make the problem bounded (in the initialisation of the algorithm).
- For the single cut problem, you can ignore the indices $k$ on the lecture slides, as there is only one subproblem being solved.

In [142]:
function benders_decomposition(ins; max_iter = 100)
    
    k = 1
    ϵ = 0.01
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.J))
    
    start = time()
    
    # TODO: initialize the main problem and add one Benders cut to make the problem bounded
    main = generate_main(ins)
    u_bar, v_bar, obj_sub_dual = generate_and_solve_dual_subproblem(ins, x_bar)
    add_benders_cut(ins, main, u_bar, v_bar)
    
    while k <= max_iter && gap > ϵ
        start_per_iter = time()
        # TODO: obtain necessary solutions
        x_bar, θ, obj_main = solve_main(ins, main)
        u_bar, v_bar, obj_sub_dual = generate_and_solve_dual_subproblem(ins, x_bar)
        
        # TODO: what is the lower bound for the objective?
        LB = obj_main
        # TODO: what about the upper bound?
        UB = min(UB, obj_main - θ + obj_sub_dual)
        
        gap = abs((UB - LB) / UB)
        stop_per_iter = time()
        println("Iter $(k): UB: $(UB), LB: $(LB), gap $(gap), time: $(round(stop_per_iter-start_per_iter, digits=2))s")
        
        if gap <= ϵ # Lower and upper bounds are (practically) same and the solution is thus optimal
            stop = time()
            println("Optimal found. \n Objective value: $(round(UB, digits=2)). \n Total time: $(round(stop-start, digits=2))s")
            return
        else
            # TODO: optimality not reached, modify the main problem for the next iteration
            add_benders_cut(ins, main, u_bar, v_bar)

            k += 1
            end
        end
    println("Maximum number of iterations exceeded")
    end

benders_decomposition (generic function with 1 method)

In [143]:
benders_decomposition(ins)

Iter 1: UB: -1263.0273793846552, LB: -63877.14746418126, gap 49.57463401569497, time: 0.21s
Iter 2: UB: -1263.0273793846552, LB: -63695.02737938465, gap 49.430440716508265, time: 0.18s
Iter 3: UB: -1299.6094936117283, LB: -32539.02737938465, gap 24.037542076547815, time: 0.14s
Iter 4: UB: -1303.8651121860473, LB: -32529.02737938465, gap 23.948153820027294, time: 0.17s
Iter 5: UB: -1303.8651121860473, LB: -32523.609493611726, gap 23.94399856982366, time: 0.17s
Iter 6: UB: -1303.8651121860473, LB: -32515.609493611726, gap 23.937862965821964, time: 0.14s
Iter 7: UB: -1303.8651121860473, LB: -32473.86511218605, gap 23.9058470916065, time: 0.2s
Iter 8: UB: -1303.8651121860473, LB: -32473.51268758718, gap 23.90557679938411, time: 0.17s
Iter 9: UB: -1303.8651121860473, LB: -32469.609493611726, gap 23.902583242812213, time: 0.18s
Iter 10: UB: -1303.8651121860473, LB: -32462.369772150185, gap 23.89703073481589, time: 0.15s
Iter 11: UB: -1303.8651121860473, LB: -32459.609493611726, gap 23.894913

In [136]:
objective_value(fullmodel)

-1327.366449586151

# Task 3: 

## Benders components (multi-cut version)

Your task is to create a version of the main problem with multiple Benders cuts being generated at each iteration, and the respective Benders cut. We refer to this version as the multi-cut version. 

Here is a bonus question that might give you ideas on how the implementation could be made more efficient: notice that the previous implementation of the dual subproblem is generating all the cut information at once, and that is why we can reutilise the function `solve_dual_subproblem(ins, x_bar)` here. Imagining that you have a number of parallel computing nodes available, can you see a way that the function `solve_dual_subproblem(ins, x_bar)` could be made more efficient? (bear in mind you are **not required** to implement or try anything in the direction of the answer to the bonus question, but only to give the question a thought!)

***My initial thought***: to generate the dual subproblems separately for each scenario $s \in S$ and solve them simultaneously using the given parallel computing nodes.

### Formulation of the initial main problem
Benders cut is generated for each scenario $s \in S$.
\begin{align*}
    \min_{x_j, \theta_s} & \ \sum_{j \in J} C_j x_j + \sum_{s \in S} \theta_s \\
    \text{s.t.: } & \sum_{j \in J} x_j \leq V   & (t)\\
    & x_j \in \{0,1\}, \ \forall j \in J \\
    & \theta_s \in \mathbb{R}, \ \forall s \in S
\end{align*}

In [145]:
## Benders decomposition: multi-cut

## Generates the main problem
function generate_main_multi(ins)
    
    J = ins.J
    S = ins.S
    V = ins.V
    C = ins.C
    
    # TODO: add your code here
    main = Model(Cbc.Optimizer)
    set_silent(main)

    @variable(main, x[J], Bin)
    @variable(main, θ[S])

    @objective(main, Min, sum(C[j] * x[j] for j in J) + sum(θ[S]))

    @constraint(main, number_of_server, sum(x[J]) <= V)

    return main  
end

# Solve the main problem
function solve_main_multi(ins, main)
    
    optimize!(main)
    
    return value.(main[:x]), value.(main[:θ]), objective_value(main)    

end

solve_main_multi (generic function with 1 method)

### Benders cut to be added
\begin{align*}
    & \theta_s \geq \sum_{j \in J} (U x_j) \overline{u}_{js} + \sum_{i \in I} H_{is} \overline{v}_{is}, \ \forall s \in S
\end{align*}

In [149]:
# Add the Benders cut, given current dual values
function add_benders_cut_multi(ins, main, u_bar, v_bar)   
    
    U = ins.U
    H = ins.H
    I = ins.I
    J = ins.J
    S = ins.S

    x = main[:x]
    θ = main[:θ]
    
    @constraint(main, [s in S], 
    # TODO: add your code here
    θ[s] >= sum(U * x[j] * u_bar[j, s] for j in J) + sum(H[i, s] * v_bar[i, s] for i in I)
    )

    return main
end

add_benders_cut_multi (generic function with 1 method)

In [152]:
function benders_decomposition_multi(ins; max_iter = 100)
    
    k = 1
    ϵ = 0.01
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.J))
    
    start = time()
    
    # TODO: initialize the main problem and add one set of Benders cuts to make the problem bounded
    main = generate_main_multi(ins)
    u_bar, v_bar, obj_sub_dual = generate_and_solve_dual_subproblem(ins, x_bar)
    add_benders_cut_multi(ins, main, u_bar, v_bar)
    
    while k <= max_iter && gap > ϵ
        start_per_iter = time()
        # TODO: obtain necessary solutions
        x_bar, θ, obj_main = solve_main_multi(ins, main)
        u_bar, v_bar, obj_sub_dual = generate_and_solve_dual_subproblem(ins, x_bar)
        
        # TODO: what is the lower bound for the objective?
        LB = obj_main
        # TODO: what about the upper bound?
        UB = min(UB, obj_main - sum(θ) + obj_sub_dual)
        
        gap = abs((UB - LB) / UB)
        stop_per_iter = time()
        println("Iter $(k): UB: $(UB), LB: $(LB), gap $(gap), time: $(round(stop_per_iter-start_per_iter, digits=2))")
        
        if gap <= ϵ # Lower and upper bounds are (practically) same and the solution is thus optimal
            stop = time()
            println("Optimal found. \n Objective value: $(round(UB, digits=2)). \n Total time: $(round(stop-start, digits=2))s")
            return
        else
            # TODO: optimality not reached, modify the main problem for the next iteration
            add_benders_cut_multi(ins, main, u_bar, v_bar)
            
            k += 1
            end
        end
    println("Maximum number of iterations exceeded")
    end

benders_decomposition_multi (generic function with 1 method)

In [153]:
benders_decomposition_multi(ins)

Iter 1: UB: -1263.0273793846625, LB: -63877.14746418113, gap 49.57463401569458, time: 0.16
Iter 2: UB: -1263.0273793846625, LB: -61701.068317767415, gap 47.85172667264562, time: 0.15
Iter 3: UB: -1263.0273793846625, LB: -32483.24397196351, gap 24.71855883899295, time: 0.35
Iter 4: UB: -1263.0273793846625, LB: -32479.933919287298, gap 24.715938109838348, time: 0.42
Iter 5: UB: -1273.1081135244865, LB: -32471.933919287298, gap 24.506030143340805, time: 0.47
Iter 6: UB: -1278.2087360342412, LB: -32470.86182525683, gap 24.403410968697205, time: 0.73
Iter 7: UB: -1278.2087360342412, LB: -32469.755129527002, gap 24.402545151011385, time: 0.78
Iter 8: UB: -1278.2087360342412, LB: -32463.10586476371, gap 24.39734313308126, time: 0.76
Iter 9: UB: -1278.2087360342412, LB: -32454.341405397172, gap 24.390486303582712, time: 0.74
Iter 10: UB: -1278.2087360342412, LB: -32453.245446661687, gap 24.38962888592894, time: 0.99
Iter 11: UB: -1327.366449586151, LB: -1329.9440775985843, gap 0.00194191137890